In [2]:
%load_ext kedro.ipython
%reload_kedro

The kedro.ipython extension is already loaded. To reload it, use:
  %reload_ext kedro.ipython
2022-10-13 18:37:31,051 - kedro.ipython - INFO - No path argument was provided. Using: /Users/jordan/Library/Mobile Documents/com~apple~CloudDocs/Code/University/Revisions/sunspots
2022-10-13 18:37:31,720 - kedro.ipython - INFO - Kedro project Sunspots
2022-10-13 18:37:31,723 - kedro.ipython - INFO - Defined global variable 'context', 'session', 'catalog' and 'pipelines'
2022-10-13 18:37:31,746 - kedro.ipython - INFO - Registered line magic 'run_viz'


In [3]:
catalog

<kedro.io.data_catalog.DataCatalog object at 0x12d811b40>

In [4]:
catalog.list()

[
    'training_dataset',
    'validation_dataset',
    '20140910Timeseries',
    'sunspot_selector',
    'image_chunk_features',
    'image_chunk_labels',
    'engineered_features',
    'fit_engineer_features.scaler',
    'feature_engineering.scaler',
    'master_table',
    'X_train',
    'X_test',
    'y_train',
    'y_test',
    'X_train_resampled',
    'y_train_resampled',
    'model',
    'target_prediction.model',
    'evaluation_targets',
    'training_targets',
    '20140910_targets',
    '20140910_region_submaps',
    'parameters',
    'params:box_size',
    'params:split_options',
    'params:split_options.target',
    'params:split_options.test_size',
    'params:modelling',
    'params:modelling.resample_options',
    'params:modelling.resample_options.mode',
    'params:modelling.resample_options.ratio',
    'params:features'
]

In [10]:
dataset = catalog.load("20140910Timeseries")

2022-10-13 18:40:28,128 - kedro.io.data_catalog - INFO - Loading data from '20140910Timeseries' (PartitionedDataSet)...


In [6]:
targets = catalog.load("20140910_targets")

2022-10-13 18:38:43,447 - kedro.io.data_catalog - INFO - Loading data from '20140910_targets' (PickleDataSet)...


In [11]:
from typing import Callable
from sunpy.map import Map
import numpy as np
from scipy import ndimage

In [13]:
def _select_region(labels, index, box_size, contour_size=None):
    """
    Return the bottom left and top right corners of the
    bounding box of a labelled image region, optionally
    contouring around the box to "fuzzily" include
    features on the edge.

    Parameters
    ----------
    labels : np.ndarray
        The integer labelled mask (produced by e.g. scipy.ndimage.label).
    index : int
        The labelled region to extract.
    box_size : int
        The size of the boxes used to define the regions.
    contour_size : Optional[int]
        The size to add onto each side of the region when defining
        the bounding box. Defauly: boxSize // 2.

    Returns
    -------
    bottomLeft, topRight : Tuple[int, int]
        The pixel coordinates of the corners of the selected region.
    """
    if contour_size is None:
        contour_size = box_size // 2

    coords = np.argwhere(labels == index)
    min_x = np.min(coords[:, 1])
    max_x = np.max(coords[:, 1])
    min_y = np.min(coords[:, 0])
    max_y = np.max(coords[:, 0])

    bottom_left = (
        (max_x + 1) * box_size + contour_size // 2,
        (max_y + 1) * box_size + contour_size // 2,
    )
    top_right = (
        min_x * box_size - contour_size // 2,
        min_y * box_size - contour_size // 2,
    )
    return bottom_left, top_right


def get_region_coords(dataset: dict[str, Callable], targets: np.ndarray, box_size):
    """
    Gets the coordinates for regions of contiguous targets for all maps in a dataset.
    """
    region_coords = {}
    for key, mask in zip(dataset.keys(), targets):
        region_coords[key] = []
        labeled_array, num_features = ndimage.label(mask)
        for index in range(1, num_features + 1):
            bounding_coords = _select_region(labeled_array, index, box_size)
            region_coords[key].append(bounding_coords)
    return region_coords

In [14]:
def _extract_region(smap, bottomLeft, topRight):
    """
    Extract a submap from the provided pixel coordinates
    (obtained from select_region).

    Parameters
    ----------
    smap : sunpy.map.Map
        The HMI map.
    bottomLeft : Tuple[int, int]
        The bottom left pixel coordinate pair.
    topRight : Tuple[int, int]
        The top right pixel coordinate pair.

    Returns
    -------
    submap : sunpy.map.Map
        The submap of the selected region.
    """
    bl = smap.wcs.pixel_to_world(*bottomLeft)
    tr = smap.wcs.pixel_to_world(*topRight)
    return smap.submap(bottom_left=bl, top_right=tr)


def extract_regions(dataset: dict[str, Callable], region_coords: dict[str, list[Map]]):
    """
    Extracts submaps from provided pixel coordinates across all maps in a
    dataset.
    """
    extractions = {}
    for key, smap, coords in zip(dataset.items(), region_coords.values()):
        # extractions[key] = []
        for coord in coords:
            bottom_left, top_right = coord
            region_submap = _extract_region(smap, bottom_left, top_right)
            extractions[f"{key}_{coord}"] = region_submap
    return extractions


In [17]:
region_coords = get_region_coords(dataset, targets, box_size=64)

In [22]:
from pprint import pprint

In [46]:
for (key, smap), coords in zip(dataset.items(), list(region_coords.values())):
    print(key)
    print(smap)
    print(coords)
    print()
    for coord in coords:
        print(coord)
    print()
    print()

hmi_ic_45s_2014_09_10_16_31_30_tai_continuum
<bound method AbstractVersionedDataSet.load of <sunspots.extras.datasets.sunpy_datasets.SunPyMapDataSet object at 0x10fb4a9e0>>
[((2320, 1936), (2032, 1584)), ((528, 1872), (432, 1776)), ((848, 1872), (752, 1776)), ((592, 1936), (496, 1840)), ((848, 2000), (752, 1904)), ((464, 2640), (368, 2480)), ((2128, 2896), (1584, 2608)), ((3472, 2704), (3376, 2608)), ((2896, 2832), (2800, 2672)), ((3024, 2832), (2928, 2736)), ((1680, 2960), (1456, 2800)), ((2128, 2896), (2032, 2800)), ((2384, 3024), (2288, 2864))]

((2320, 1936), (2032, 1584))
((528, 1872), (432, 1776))
((848, 1872), (752, 1776))
((592, 1936), (496, 1840))
((848, 2000), (752, 1904))
((464, 2640), (368, 2480))
((2128, 2896), (1584, 2608))
((3472, 2704), (3376, 2608))
((2896, 2832), (2800, 2672))
((3024, 2832), (2928, 2736))
((1680, 2960), (1456, 2800))
((2128, 2896), (2032, 2800))
((2384, 3024), (2288, 2864))


hmi_ic_45s_2014_09_10_16_34_30_tai_continuum
<bound method AbstractVersioned